In [1]:
import numpy as np
import pandas as pd
import foolbox

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop, adam
from keras import backend as K

from cleverhans.attacks import FastGradientMethod, SaliencyMapMethod
from cleverhans.utils_tf import model_train , model_eval , batch_eval, model_argmax
from cleverhans.attacks_tf import jacobian_graph
from cleverhans.utils import other_classes
from cleverhans.utils_keras import KerasModelWrapper

import tensorflow as tf
from tensorflow.python.platform import flags

from sklearn.multiclass import OneVsRestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC

from sklearn.metrics import accuracy_score, roc_curve, auc, f1_score
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import matplotlib.pyplot as plt
plt.style.use('bmh')

K.set_learning_phase(1)

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/usr/local/lib/python3.5/dist-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
FLAGS = flags.FLAGS

flags.DEFINE_integer('nb_epochs', 20, 'Number of epochs to train model')
flags.DEFINE_integer('batch_size', 128, 'Size of training batches')
flags.DEFINE_integer('learning_rate', 0.1, 'Learning rate for training')
flags.DEFINE_integer('nb_classes', 2, 'Number of classification classes')
flags.DEFINE_integer('source_samples', 10, 'Nb of test set examples to attack')

In [3]:
print()
print()
print("================================= Start of preprocessing stage ==============================")

names = ['duration', 'protocol', 'service', 'flag', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell', 'su_attempted', 'num_root', 'num_file_creations', 'num_shells', 'num_access_files', 'num_outbound_cmds', 'is_host_login', 'is_guest_login', 'count', 'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate', 'attack_type', 'other']

df_train = pd.read_csv('./NSL_KDD/KDDTrain+.txt', names=names, header=None)
df_test = pd.read_csv('./NSL_KDD/KDDTest+.txt', names=names, header=None)
print("Initial training and test data shapes: ", df_train.shape, df_test.shape)

full = pd.concat([df_train, df_test])
assert full.shape[0] == df_train.shape[0] + df_test.shape[0]

full['label'] = full['attack_type']



================================= Start of preprocessing stage ==============================
Initial training and test data shapes:  (125973, 43) (22544, 43)


In [4]:
full.head()

,duration,protocol,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,attack_type,other,label
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal,20,normal
1,0,udp,other,SF,146,0,0,0,0,0,...,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal,15,normal
2,0,tcp,private,S0,0,0,0,0,0,0,...,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,19,neptune
3,0,tcp,http,SF,232,8153,0,0,0,0,...,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal,21,normal
4,0,tcp,http,SF,199,420,0,0,0,0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal,21,normal


In [5]:
# DoS Attacks
full.loc[full.label == 'neptune', 'label'] = 'other'
full.loc[full.label == 'back', 'label'] = 'other'
full.loc[full.label == 'land', 'label'] = 'other'
full.loc[full.label == 'pod', 'label'] = 'other'
full.loc[full.label == 'smurf', 'label'] = 'other'
full.loc[full.label == 'teardrop', 'label'] = 'other'
full.loc[full.label == 'mailbomb', 'label'] = 'other'
full.loc[full.label == 'processtable', 'label'] = 'other'
full.loc[full.label == 'udpstorm', 'label'] = 'other'
full.loc[full.label == 'apache2', 'label'] = 'other'
full.loc[full.label == 'worm', 'label'] = 'other'

# User-to-root (U2R)
full.loc[full.label == 'buffer_overflow', 'label'] = 'other'
full.loc[full.label == 'loadmodule', 'label'] = 'other'
full.loc[full.label == 'perl', 'label'] = 'other'
full.loc[full.label == 'rootkit', 'label'] = 'other'
full.loc[full.label == 'sqlattack', 'label'] = 'other'
full.loc[full.label == 'xterm', 'label'] = 'other'
full.loc[full.label == 'ps', 'label'] = 'other'

# Remote-to-local (R2L)
full.loc[full.label == 'ftp_write', 'label'] = 'other'
full.loc[full.label == 'guess_passwd', 'label'] = 'other'
full.loc[full.label == 'imap', 'label'] = 'other'
full.loc[full.label == 'multihop', 'label'] = 'other'
full.loc[full.label == 'phf', 'label'] = 'other'
full.loc[full.label == 'spy', 'label'] = 'other'
full.loc[full.label == 'warezclient', 'label'] = 'other'
full.loc[full.label == 'warezmaster', 'label'] = 'other'
full.loc[full.label == 'xlock', 'label'] = 'other'
full.loc[full.label == 'xsnoop', 'label'] = 'other'
full.loc[full.label == 'snmpgetattack', 'label'] = 'other'
full.loc[full.label == 'httptunnel', 'label'] = 'other'
full.loc[full.label == 'snmpguess', 'label'] = 'other'
full.loc[full.label == 'sendmail', 'label'] = 'other'
full.loc[full.label == 'named', 'label'] = 'other'

# Probe attacls
full.loc[full.label == 'satan', 'label'] = 'other'
full.loc[full.label == 'ipsweep', 'label'] = 'other'
full.loc[full.label == 'nmap', 'label'] = 'other'
full.loc[full.label == 'portsweep', 'label'] = 'other'
full.loc[full.label == 'saint', 'label'] = 'other'
full.loc[full.label == 'mscan', 'label'] = 'other'

In [6]:
full.head()

,duration,protocol,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,attack_type,other,label
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,...,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal,20,normal
1,0,udp,other,SF,146,0,0,0,0,0,...,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal,15,normal
2,0,tcp,private,S0,0,0,0,0,0,0,...,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,19,other
3,0,tcp,http,SF,232,8153,0,0,0,0,...,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal,21,normal
4,0,tcp,http,SF,199,420,0,0,0,0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal,21,normal


In [7]:
full = full.drop(['other', 'attack_type'], axis=1)
print("Unique labels", full.label.unique())
print(full.head())

Unique labels ['normal' 'other']
   duration protocol   service flag  src_bytes  dst_bytes  land  \
0         0      tcp  ftp_data   SF        491          0     0   
1         0      udp     other   SF        146          0     0   
2         0      tcp   private   S0          0          0     0   
3         0      tcp      http   SF        232       8153     0   
4         0      tcp      http   SF        199        420     0   

   wrong_fragment  urgent  hot   ...    dst_host_srv_count  \
0               0       0    0   ...                    25   
1               0       0    0   ...                     1   
2               0       0    0   ...                    26   
3               0       0    0   ...                   255   
4               0       0    0   ...                   255   

   dst_host_same_srv_rate  dst_host_diff_srv_rate  \
0                    0.17                    0.03   
1                    0.00                    0.60   
2                    0.10       

In [8]:
# Generate One - Hot encoding
full2 = pd.get_dummies(full, drop_first=False)
full2.head()

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_RSTOS0,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH,label_normal,label_other
0,0,491,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
1,0,146,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1
3,0,232,8153,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,1,0
4,0,199,420,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,1,0


In [9]:
full2.columns

Index(['duration', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment',
       'urgent', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised',
       ...
       'flag_RSTOS0', 'flag_RSTR', 'flag_S0', 'flag_S1', 'flag_S2', 'flag_S3',
       'flag_SF', 'flag_SH', 'label_normal', 'label_other'],
      dtype='object', length=124)

In [10]:
# Separate training and test sets again
features = list(full2.columns[:-2])   # Due to One-Hot encoding
features

['duration',
 'src_bytes',
 'dst_bytes',
 'land',
 'wrong_fragment',
 'urgent',
 'hot',
 'num_failed_logins',
 'logged_in',
 'num_compromised',
 'root_shell',
 'su_attempted',
 'num_root',
 'num_file_creations',
 'num_shells',
 'num_access_files',
 'num_outbound_cmds',
 'is_host_login',
 'is_guest_login',
 'count',
 'srv_count',
 'serror_rate',
 'srv_serror_rate',
 'rerror_rate',
 'srv_rerror_rate',
 'same_srv_rate',
 'diff_srv_rate',
 'srv_diff_host_rate',
 'dst_host_count',
 'dst_host_srv_count',
 'dst_host_same_srv_rate',
 'dst_host_diff_srv_rate',
 'dst_host_same_src_port_rate',
 'dst_host_srv_diff_host_rate',
 'dst_host_serror_rate',
 'dst_host_srv_serror_rate',
 'dst_host_rerror_rate',
 'dst_host_srv_rerror_rate',
 'protocol_icmp',
 'protocol_tcp',
 'protocol_udp',
 'service_IRC',
 'service_X11',
 'service_Z39_50',
 'service_aol',
 'service_auth',
 'service_bgp',
 'service_courier',
 'service_csnet_ns',
 'service_ctf',
 'service_daytime',
 'service_discard',
 'service_domain',
 '

In [11]:
y_train = np.array(full2[0: df_train.shape[0]][['label_normal', 'label_other']])
X_train = full2[0: df_train.shape[0]][features]

In [12]:
y_train

array([[1, 0],
       [1, 0],
       [0, 1],
       ...,
       [1, 0],
       [0, 1],
       [1, 0]], dtype=uint8)

In [13]:
y_test = np.array(full2[df_train.shape[0]: ][['label_normal', 'label_other']])
X_test = full2[df_train.shape[0]: ][features]

In [14]:
X_test

,duration,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,...,flag_REJ,flag_RSTO,flag_RSTOS0,flag_RSTR,flag_S0,flag_S1,flag_S2,flag_S3,flag_SF,flag_SH
0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,2,12983,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,0,20,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,1,0,15,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
5,0,267,14515,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
6,0,1022,387,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
7,0,129,174,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
8,0,327,467,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
9,0,26,157,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0


In [15]:
# Scale data
scaler = MinMaxScaler().fit(X_train)
X_train_scaled = np.array(scaler.transform(X_train))
X_test_scaled = np.array(scaler.transform(X_test))

In [16]:
X_train_scaled

array([[0.00000000e+00, 3.55806412e-07, 0.00000000e+00, ...,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 1.05799870e-07, 0.00000000e+00, ...,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [0.00000000e+00, 1.61670897e-06, 2.93143779e-07, ...,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 1.09423153e-07, 0.00000000e+00, ...,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00]])

In [17]:
# Generate label encoding for Logistic regression
labels = full.label.unique()
le = LabelEncoder()
le.fit(labels)

LabelEncoder()

In [18]:
y_full = le.transform(full.label)
y_train_l = y_full[0: df_train.shape[0]]
y_test_l = y_full[df_train.shape[0]: ]

In [19]:
y_train_l

array([0, 0, 1, ..., 0, 1, 0])

In [20]:
print("Training dataset shape", X_train_scaled.shape, y_train.shape)
print("Test dataset shape", X_test_scaled.shape, y_test.shape)
print("Label encoder y shape", y_train_l.shape, y_test_l.shape)

Training dataset shape (125973, 122) (125973, 2)
Test dataset shape (22544, 122) (22544, 2)
Label encoder y shape (125973,) (22544,)


In [21]:
print("================================= End of preprocessing stage ================================")
print()
print()

================================= End of preprocessing stage ================================




In [22]:
def mlp_model():
    """
    Generate a Multilayer Perceptron model
    """
    model = Sequential()
    model.add(Dense(256, activation='relu', input_shape=(X_train_scaled.shape[1], )))
    model.add(Dropout(0.2))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(FLAGS.nb_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    model.summary()
    return model

In [23]:
def evaluate():
    """
    Model evaluation function
    """
    eval_params = {'batch_size': FLAGS.batch_size}
    accuracy = model_eval(sess, x, y, predictions, X_test_scaled, y_test, args=eval_params)
    print("Test accuracy on legitimate test samples: " + str(accuracy))

In [24]:
# Tensorflow placeholder variables
x = tf.placeholder(tf.float32, shape=(None, X_train_scaled.shape[1]))
y = tf.placeholder(tf.float32, shape=(None, FLAGS.nb_classes))

In [25]:
tf.set_random_seed(42)
model = mlp_model()
sess = tf.Session()
# K.set_session(sess)
predictions = model(x)
init = tf.global_variables_initializer()
sess.run(init)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               31488     
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 130       
Total para

In [26]:
# Train the model
train_params = {'nb_epochs': FLAGS.nb_epochs, 'batch_size': 64, 'learning_rate': 0.01, 'verbose': 1}

In [27]:
model_train(sess, x, y, predictions, X_train_scaled, y_train, evaluate=evaluate, args=train_params)

/usr/local/lib/python3.5/dist-packages/cleverhans/utils_tf.py:422: UserWarning: This function is deprecated and will be removed on or after 2019-04-05. Switch to cleverhans.train.train.
  warnings.warn("This function is deprecated and will be removed on or after"
/usr/local/lib/python3.5/dist-packages/cleverhans/utils_tf.py:37: UserWarning: This function is deprecated and will be removed on or after 2019-04-05. Switch to cleverhans.train.train.
  warnings.warn("This function is deprecated and will be removed on or after"
/usr/local/lib/python3.5/dist-packages/cleverhans/compat.py:130: UserWarning: Running on tensorflow version 1.4.1. Support for this version in CleverHans is deprecated and may be removed on or after 2019-01-26
  warnings.warn(warning)
/usr/local/lib/python3.5/dist-packages/cleverhans/compat.py:33: UserWarning: Running on tensorflow version 1.4.1. Support for this version in CleverHans is deprecated and may be removed on or after 2019-01-26
  warnings.warn(warning)
[INF

Test accuracy on legitimate test samples: 0.7865951029098651


[INFO 2019-02-08 21:39:59,736 cleverhans] Epoch 1 took 5.784933090209961 seconds


Test accuracy on legitimate test samples: 0.7772799858055358


[INFO 2019-02-08 21:40:05,948 cleverhans] Epoch 2 took 5.808060884475708 seconds


...

[INFO 2019-02-08 21:41:44,040 cleverhans] Epoch 18 took 5.804677724838257 seconds


Test accuracy on legitimate test samples: 0.7837562100780695


[INFO 2019-02-08 21:41:50,160 cleverhans] Epoch 19 took 5.784427881240845 seconds
[INFO 2019-02-08 21:41:50,477 cleverhans] Completed model training.


Test accuracy on legitimate test samples: 0.7848651525904897


True

In [28]:
# model.save("first_model_1.h5")

In [29]:
a = []
for i in range(X_test_scaled.shape[0]):
    a.append(model.predict_classes(X_test_scaled[i].reshape(1,122), verbose=0)[0])

In [30]:
a = np.array(a)
unique_elements, counts_elements = np.unique(a, return_counts=True)
print("Frequency of unique values of the said array:")
print(np.asarray((unique_elements, counts_elements)))

Frequency of unique values of the said array:
[[    0     1]
 [14203  8341]]


In [31]:
# Generate adversarial samples for all test datapoints
source_samples = X_test_scaled.shape[0]

In [32]:
# Craft adversarial examples using Fast Gradient Sign Method (FGSM)
models = KerasModelWrapper(model)
fgsm = FastGradientMethod(models, sess=sess)
fgsm_params = {'eps': 0.3, 'clip_min': 0., 'clip_max': 1.}

In [33]:
adv_x = fgsm.generate(x, **fgsm_params)
# adv_x = tf.stop_gradient(adv_x)
X_test_adv, = batch_eval(sess, [x], [adv_x], [X_test_scaled])
# preds_adv = model(adv_x)

/usr/local/lib/python3.5/dist-packages/cleverhans/compat.py:33: UserWarning: Running on tensorflow version 1.4.1. Support for this version in CleverHans is deprecated and may be removed on or after 2019-01-26
  warnings.warn(warning)
/usr/local/lib/python3.5/dist-packages/cleverhans/compat.py:130: UserWarning: Running on tensorflow version 1.4.1. Support for this version in CleverHans is deprecated and may be removed on or after 2019-01-26
  warnings.warn(warning)
/usr/local/lib/python3.5/dist-packages/cleverhans/utils_tf.py:294: UserWarning: batch_eval has moved to cleverhans.evaluation. batch_eval will be removed from utils_tf on or after 2019-03-09.
  warnings.warn("batch_eval has moved to cleverhans.evaluation. "


InvalidArgumentError: assertion failed: [] [Condition x <= y did not hold element-wise:x (Placeholder:0) = ] [[2.3305678e-05 4.34794e-09 4.12233447e-08]...] [y (Cast_2/x:0) = ] [1]
	 [[Node: assert_less_equal/Assert/AssertGuard/Assert = Assert[T=[DT_STRING, DT_STRING, DT_FLOAT, DT_STRING, DT_FLOAT], summarize=3, _device="/job:localhost/replica:0/task:0/device:CPU:0"](assert_less_equal/Assert/AssertGuard/Assert/Switch, assert_less_equal/Assert/AssertGuard/Assert/data_0, assert_less_equal/Assert/AssertGuard/Assert/data_1, assert_less_equal/Assert/AssertGuard/Assert/Switch_1, assert_less_equal/Assert/AssertGuard/Assert/data_3, assert_less_equal/Assert/AssertGuard/Assert/Switch_2)]]

Caused by op 'assert_less_equal/Assert/AssertGuard/Assert', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykern

In [ ]:
eval_par = {'batch_size': 64}

In [ ]:
# acc = model_eval(sess, x, y, preds_adv, X_test_scaled, y_test, args=eval_par)

In [ ]:
# acc